<a href="https://colab.research.google.com/github/happy-jihye/Natural-Language-Processing/blob/main/Simple_RNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple RNN model

- Pytorch / TorchText
- RNN network를 사용한 간단한 Sentiment Analysis 예제
> - 2021/03/21 Happy-jihye
> - **Reference** : [pytorch-sentiment-analysis/1 - Simple Sentiment Analysis](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb)


## 1. Preparing Data


#### 1) Text/Label
- **[spaCy](https://spacy.io/)** : nlp를 쉽게 할 수 있도록 도와주는 파이썬 패키지로, tokenizaing, parsing, pos tagging 등을 지원
- **[Field](https://pytorch.org/text/_modules/torchtext/data/field.html)** 에 정의된 내용을 기반으로 영어를 token화

In [ ]:
!pip install -U torchtext==0.8.0

In [28]:
%%capture
!python -m spacy download en

In [29]:
import torch
from torchtext.legacy import data
# torchtext.legacy : https://stackoverflow.com/questions/66516388/attributeerror-module-torchtext-data-has-no-attribute-field

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en')
LABEL = data.LabelField(dtype = torch.float)

#### 2) IMDb Dataset
- 5만개의 영화 리뷰로 구성된 IMDB dataset을 다운로드 받은 후, 이전에 정의한 Field를 사용해서 데이터를 처리(TEXT, LABEL)
- torchtext.datasets 의 [IMDB](https://pytorch.org/text/stable/datasets.html#imdb) 객체로 train data와 test data을 split

In [37]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [38]:
print(f'training examples 수 : {len(train_data)}')
print(f'testing examples 수 : {len(test_data)}')
print(vars(train_data.examples[0]))

training examples 수 : 25000
testing examples 수 : 25000
{'text': ['This', 'is', 'a', 'neat', 'little', 'crime', 'drama', 'which', 'packs', 'a', 'lot', 'into', 'its', '65', 'minute', 'running', 'time', '.', 'It', 'has', 'all', 'the', 'right', 'ingredients', '-', 'a', 'mystery', 'corpse', ',', 'a', 'weary', 'middle', '-', 'aged', 'cop', 'Corrigan', '(', 'Walter', 'Kinsella', ')', 'and', 'his', 'rookie', 'sidekick', 'Tobin', '(', 'John', 'Miles', ')', ',', 'a', 'shadowy', 'killer', 'on', 'the', 'loose', 'and', 'even', 'love', 'interest', 'for', 'the', 'Tobin', 'in', 'the', 'shape', 'of', 'a', 'female', 'botanist', 'Mary', '(', 'Patricia', 'Wright', ')', 'who', 'helps', 'solve', 'the', 'crime', '.', 'There', "'s", 'also', 'a', 'terrific', 'shoot', '-', 'out', 'finale', 'which', 'takes', 'place', 'in', 'a', 'stone', 'cutters', 'yard.<br', '/><br', '/>Watch', 'out', 'for', 'a', 'terrific', 'goof', 'near', 'the', 'start', 'of', 'this', 'movie', 'where', 'Lt', '.', 'Corrigan', 'refers', 'to', '

- IMDb dataset은 train/test data만 있고, validation set이 없음

  따라서 train dataset을 split해서 validation dataset 을 만들기!!
  (split 함수의 default split_ratio = 0.7)

In [39]:
import random

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [40]:
print(f'training examples 수 : {len(train_data)}')
print(f'validations examples 수 : {len(valid_data)}')
print(f'testing examples 수 : {len(test_data)}')

training examples 수 : 17500
validations examples 수 : 7500
testing examples 수 : 25000


#### 3) Build Vocabulary
- one-hot encoding 방식을 사용해서 단어들을 indexing 하기
- training dataset에 있는 단어들은 10만개가 넘는데, 이 모든 단어들에 대해 indexing을 하면 one-hot vector의 dimension이 10만개가 됨.. 즉, 너무나도 차원이 크기 때문에 연산하기 안좋을 수도 있음
  - 따라서 자주 나오는 단어들만 가지고 훈련을 하기! 이 예제에서는 **25,000 words**를 사용
  - ex) "This film is great and I love it" 라는 문장에서 "love"라는 단어가 vocabulary에 없다면, "This film is great and I <unk> it"로 문장을 학습시킬 것
  

In [ ]:
MAX_VOCAB_SIZE = 